In [1]:
import json
import os
from pprint import pprint

import pyomo.environ as pe
import pyomo.opt as po
import yaml

In [2]:
cfl_traditional_filename = os.path.join(
    #'data',
    'cfl.yaml'
)
cfl_blk_demand_filename = os.path.join(
    #'data',
    'cfl_blk_by_demand.json'
)
cfl_blk_supply_filename = os.path.join(
    #'data',
    'cfl_blk_by_supply.json'
)

'cfl.yaml'

# Setup Solver

In [3]:
solver = po.SolverFactory('glpk')

# Capacitated Facility Location
To demonstrate blocks, let's setup and solve an instance of the CFL model.

Sets:
- $I$: supply sites, indexed by $i$
- $J$: demand sites, indexed by $j$

Parameters:
- $s_i$: supply capacity of supply site $i$
- $d_j$: demand required by demand site $j$
- $f_i$: fixed cost to open supply site $i$
- $c_{ij}$: variable cost to transport from supply site $i$ to demand site $j$

Variables:
- $x_{ij}$ - quantity of product to ship from supply site $i$ to demand site $j$
- $y_i$ - 0/1 decision variable indicating that supply site $i$ is producing

Model:
$$
\begin{alignat*}{3}
\text{minimize  }  & \sum_{i \in I} f_i y_i + \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \\
\text{subject to  }
& \sum_{i \in I} x_{ij} \ge d_j && \forall j \in J \\
& \sum_{j \in J} x_{ij} \le s_i y_i && \forall i \in I \\
& x \in \mathbb{R}_+^{|I| \times |J|} && \\
& y \in \{0, 1\}^{|I|} && \\
\end{alignat*}
$$

# Implementation without Blocks

In [4]:
with open(cfl_traditional_filename) as fh:
    data = yaml.load(fh, Loader=yaml.FullLoader)
pprint(data)

{'demand': {'a': 80, 'b': 270, 'c': 250, 'd': 160, 'e': 180},
 'demand_sites': ['a', 'b', 'c', 'd', 'e'],
 'fixed_cost': {'1': 1000, '2': 1000, '3': 1000},
 'supply': {'1': 500, '2': 500, '3': 500},
 'supply_sites': ['1', '2', '3'],
 'variable_cost': {('1', 'a'): 4,
                   ('1', 'b'): 5,
                   ('1', 'c'): 6,
                   ('1', 'd'): 8,
                   ('1', 'e'): 10,
                   ('2', 'a'): 6,
                   ('2', 'b'): 4,
                   ('2', 'c'): 3,
                   ('2', 'd'): 5,
                   ('2', 'e'): 8,
                   ('3', 'a'): 9,
                   ('3', 'b'): 7,
                   ('3', 'c'): 4,
                   ('3', 'd'): 3,
                   ('3', 'e'): 4}}


In [5]:
model = pe.ConcreteModel("Traditional")

# sets
model.I = pe.Set(initialize=data['supply_sites'])
model.J = pe.Set(initialize=data['demand_sites'])

# parameters
model.s = pe.Param(model.I, initialize=data['supply'])
model.d = pe.Param(model.J, initialize=data['demand'])
model.f = pe.Param(model.I, initialize=data['fixed_cost'])
model.c = pe.Param(model.I, model.J, initialize=data['variable_cost'])

# variables
model.x = pe.Var(model.I, model.J, domain=pe.NonNegativeReals)
model.y = pe.Var(model.I, domain=pe.Binary)

# constraints
def con_satisfaction(model, j):
    return sum(model.x[i, j] for i in model.I) >= model.d[j]
model.con_satisfaction = pe.Constraint(model.J, rule=con_satisfaction)

def con_transportation(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.s[i] * model.y[i]
model.con_transportation = pe.Constraint(model.I, rule=con_transportation)

# objective
def obj_min_cost(model):
    return sum(model.f[i] * model.y[i] for i in model.I)\
        + sum(model.c[i, j] * model.x[i, j] for i in model.I for j in model.J)
model.obj_min_cost = pe.Objective(sense=pe.minimize, rule=obj_min_cost)

In [6]:
result = solver.solve(model)
model.display()

Model Traditional

  Variables:
    x : Size=15, Index=x_index
        Key        : Lower : Value : Upper : Fixed : Stale : Domain
        ('1', 'a') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('1', 'b') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('1', 'c') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('1', 'd') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('1', 'e') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('2', 'a') :     0 :  80.0 :  None : False : False : NonNegativeReals
        ('2', 'b') :     0 : 270.0 :  None : False : False : NonNegativeReals
        ('2', 'c') :     0 : 150.0 :  None : False : False : NonNegativeReals
        ('2', 'd') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('2', 'e') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('3', 'a') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('3

# Implementation with Blocks Indexed by Demand Sites

In [7]:
with open(cfl_blk_demand_filename) as fh:
    data = json.load(fh)
pprint(data)

{'demand_data': {'a': {'demand': 80, 'variable_cost': {'1': 4, '2': 6, '3': 9}},
                 'b': {'demand': 270,
                       'variable_cost': {'1': 5, '2': 4, '3': 7}},
                 'c': {'demand': 250,
                       'variable_cost': {'1': 6, '2': 3, '3': 4}},
                 'd': {'demand': 160,
                       'variable_cost': {'1': 8, '2': 5, '3': 3}},
                 'e': {'demand': 180,
                       'variable_cost': {'1': 10, '2': 8, '3': 4}}},
 'demand_sites': ['a', 'b', 'c', 'd', 'e'],
 'fixed_cost': {'1': 1000, '2': 1000, '3': 1000},
 'supply': {'1': 500, '2': 500, '3': 500},
 'supply_sites': ['1', '2', '3']}


In [8]:
model = pe.ConcreteModel("Blocks (by Demand Site)")

# sets
model.I = pe.Set(initialize=data['supply_sites'])
model.J = pe.Set(initialize=data['demand_sites'])

# parameters (not indexed in J)
model.s = pe.Param(model.I, initialize=data['supply'])
model.f = pe.Param(model.I, initialize=data['fixed_cost'])

# variables (not indexed in J)
model.y = pe.Var(model.I, domain=pe.Binary)

# blocks (indexed in J)
def blk_demand(block, j):
    blk_data = data['demand_data'][j]
    I = block.model().I # borrow the set of supply sites from the overarching model
    block.d = pe.Param(initialize=blk_data['demand'])
    block.c = pe.Param(I, initialize=blk_data['variable_cost'])
    block.x = pe.Var(I, domain=pe.NonNegativeReals)
    block.con_satisfaction = pe.Constraint(expr=(sum(block.x[i] for i in I) >= block.d))
    block.variable_cost = sum(block.c[i] * block.x[i] for i in I)
model.blk_demand = pe.Block(model.J, rule=blk_demand)

# constraints (not indexed in J)
def con_transportation(model, i):
    return sum(model.blk_demand[j].x[i] for j in model.J) <= model.s[i] * model.y[i]
model.con_transportation = pe.Constraint(model.I, rule=con_transportation)

# objective
def obj_min_cost(model):
    return sum(model.f[i] * model.y[i] for i in model.I)\
        + sum(model.blk_demand[j].variable_cost for j in model.J)
model.obj_min_cost = pe.Objective(sense=pe.minimize, rule=obj_min_cost)

In [9]:
result = solver.solve(model)
model.display()

Model 'Blocks (by Demand Site)'

  Variables:
    y : Size=3, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :     1 : False : False : Binary
          2 :     0 :   1.0 :     1 : False : False : Binary
          3 :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    obj_min_cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 5610.0

  Constraints:
    con_transportation : Size=3
        Key : Lower : Body  : Upper
          1 :  None :   0.0 :   0.0
          2 :  None :   0.0 :   0.0
          3 :  None : -60.0 :   0.0

  Blocks:
    Block blk_demand[a]
    
      Variables:
        x : Size=3, Index=I
            Key : Lower : Value : Upper : Fixed : Stale : Domain
              1 :     0 :   0.0 :  None : False : False : NonNegativeReals
              2 :     0 :  80.0 :  None : False : False : NonNegativeReals
              3 :     0 :   0.0 :  None : False : False : NonNegati

# Implementation with Blocks Indexed by Supply Sites

In [10]:
with open(cfl_blk_supply_filename) as fh:
    data = json.load(fh)
pprint(data)

{'demand': {'a': 80, 'b': 270, 'c': 250, 'd': 160, 'e': 180},
 'demand_sites': ['a', 'b', 'c', 'd', 'e'],
 'supply_data': {'1': {'fixed_cost': 1000,
                       'supply': 500,
                       'variable_cost': {'a': 4,
                                         'b': 5,
                                         'c': 6,
                                         'd': 8,
                                         'e': 10}},
                 '2': {'fixed_cost': 1000,
                       'supply': 500,
                       'variable_cost': {'a': 6,
                                         'b': 4,
                                         'c': 3,
                                         'd': 5,
                                         'e': 8}},
                 '3': {'fixed_cost': 1000,
                       'supply': 500,
                       'variable_cost': {'a': 9,
                                         'b': 7,
                                         'c': 4,
         

In [11]:
model = pe.ConcreteModel("Blocks (by Supply Site)")

# sets
model.I = pe.Set(initialize=data['supply_sites'])
model.J = pe.Set(initialize=data['demand_sites'])

# parameters (not indexed in I)
model.d = pe.Param(model.J, initialize=data['demand'])

# blocks (indexed in I)
def blk_supply(block, i):
    blk_data = data['supply_data'][i]
    J = block.model().J # borrow the set of demand sites from the overarching model
    block.c = pe.Param(J, initialize=blk_data['variable_cost'])
    block.f = pe.Param(initialize=blk_data['fixed_cost'])
    block.s = pe.Param(initialize=blk_data['supply'])
    block.x = pe.Var(J, domain=pe.NonNegativeReals)
    block.y = pe.Var(domain=pe.Binary)
    transportation = sum(block.x[j] for j in J)
    block.con_transportation = pe.Constraint(expr=(transportation <= block.s * block.y))
    block.obj = block.f * block.y + sum(block.c[j] * block.x[j] for j in J)
model.blk_supply = pe.Block(model.I, rule=blk_supply)

# constraints (not indexed in I)
def con_satisfaction(model, j):
    return sum(model.blk_supply[i].x[j] for i in model.I) == model.d[j]
model.con_satisfaction = pe.Constraint(model.J, rule=con_satisfaction)

# objective
def obj_min_cost(model):
    return sum(model.blk_supply[i].obj for i in model.I)
model.obj_min_cost = pe.Objective(sense=pe.minimize, rule=obj_min_cost)

In [12]:
result = solver.solve(model)
model.display()

Model 'Blocks (by Supply Site)'

  Variables:
    None

  Objectives:
    obj_min_cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 5610.0

  Constraints:
    con_satisfaction : Size=5
        Key : Lower : Body  : Upper
          a :  80.0 :  80.0 :  80.0
          b : 270.0 : 270.0 : 270.0
          c : 250.0 : 250.0 : 250.0
          d : 160.0 : 160.0 : 160.0
          e : 180.0 : 180.0 : 180.0

  Blocks:
    Block blk_supply['1']
    
      Variables:
        x : Size=5, Index=J
            Key : Lower : Value : Upper : Fixed : Stale : Domain
              a :     0 :   0.0 :  None : False : False : NonNegativeReals
              b :     0 :   0.0 :  None : False : False : NonNegativeReals
              c :     0 :   0.0 :  None : False : False : NonNegativeReals
              d :     0 :   0.0 :  None : False : False : NonNegativeReals
              e :     0 :   0.0 :  None : False : False : NonNegativeReals
        y : Size=1, Index=Non